In [44]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
df = pd.read_csv(r"properties.csv")

In [45]:
from datetime import datetime, timedelta
def extract_days(listed_text):
    match = re.search(r'(\d+)\s+(\w+)', listed_text)
    if match:
        value = int(match.group(1))
        unit = match.group(2).lower()
        if unit == 'day' or unit == 'days':
            return value
        elif unit == 'month' or unit == 'months':
            if value > 6:
                return "<6" 
            return value * 30 
    return None



In [46]:
df['Days Listed'] = df['Listed on'].apply(extract_days)

In [47]:
def segment_listed(days):
    if days is None:
        return "Unknown"
    elif days == "<6":
        return "Listed more than 6 months ago"
    elif days <= 7:
        return "Newly Listed"
    elif days <= 30:
        return "Recently Listed"
    elif days <= 90:
        return "Listed within 3 months"
    elif days <= 180:
        return "Listed more than 6 months ago"

df['Listed Segmentation'] = df['Days Listed'].apply(segment_listed)


In [48]:
df

,Reference,Property Type,Ad Type,Bedrooms,Bathrooms,Area (sqft),Details,Service Type,Address,Price (AED),...,Agent ID,Latitude,Longitude,Location Segment,Area Segment,Price Segment,Unnamed: 17,Listed on,Days Listed,Listed Segmentation
0,insight-4993939,Apartment,VERIFIED,1,1,843,WITH BALCONY | 1B/R | SKYCOURT | POOL VIEW,PREMIUM,"Skycourts Tower F, Skycourts Towers, Dubai Lan...",380000.0,...,2f2d1393-2c25-40fa-9cf7-f5b0801f0075,25.090079,55.385788,NaN,Small,"(0.0, 1000000.0]",NaN,Listed 11 days ago,11.0,Recently Listed
1,WIL-S-5498,Villa,VERIFIED,2,2,2967,2 BEDROOM | EXTENDED & UPGRADED | CORNER PLOT,PREMIUM,"Mediterranean Villas, Jumeirah Village Triangl...",3300000.0,...,ca212376-24d4-42ff-a81b-8d9c2933ac0c,25.047288,55.190023,Jumeirah Village Triangle,Large,"(3000000.0, inf]",NaN,Listed 6 months ago,180.0,Listed more than 6 months ago
2,luxehomes-4946277,Apartment,VERIFIED,2,3,1709,2-Bedroom + Study | Vacant | Palm Sea View,PREMIUM,"Oceana Atlantic, Oceana, Palm Jumeirah, Dubai",5500000.0,...,ebee2af7-7b22-4d11-9746-8e15411f5ff6,25.111400,55.137291,Palm Jumeirah,Medium,"(3000000.0, inf]",NaN,Listed 16 days ago,16.0,Recently Listed
3,PB-S-1641,Townhouse,VERIFIED,3,4,2235,3 Bed Type 1 | Single Row | Large Plot | Rented,PREMIUM,"Hayat Townhouses, Town Square, Dubai",2000000.0,...,22be22d1-1ee6-4eb6-a6b0-603f07356406,25.001965,55.298557,NaN,Large,"(1000000.0, 2000000.0]",NaN,Listed 1 month ago,30.0,Recently Listed
4,eandt-4966526,Villa,VERIFIED,4,4,2445,Serene desert view | Single Row | Bright,PREMIUM,"Sun, Arabian Ranches 3, Dubai",3000000.0,...,97f12ec3-dc4a-4a74-95c5-f489361cefcf,25.068276,55.321369,NaN,Large,"(2000000.0, 3000000.0]",NaN,Listed 12 days ago,12.0,Recently Listed
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1889,ENRE-FA-CT-S-0032,Apartment,VERIFIED,1,2,875,1 Bed | Stunning Views | Prime Location,PREMIUM,"Cayan Tower, Dubai Marina, Dubai",1950000.0,...,98d4d654-8f42-44a4-83e1-9655e813c9ae,25.086716,55.145145,Palm Jumeirah,Small,"(1000000.0, 2000000.0]",NaN,Listed 12 days ago,12.0,Recently Listed
1890,SALE015290,Apartment,UNVERIFIED,2,3,"2,237",Marina Views / Upgraded / Stunning,PREMIUM,"Sadaf 4, Sadaf, Jumeirah Beach Residence, Dubai",3900000.0,...,4ca11738-4a26-4ccf-ba18-913cc80e0131,25.080577,55.137152,Palm Jumeirah,Large,"(3000000.0, inf]",NaN,Listed 6 days ago,6.0,Newly Listed
1891,MZY-S-2061,Apartment,UNVERIFIED,4,5,"3,100",Sea view Full Upgraded 4BHK-2 Parking - Al Mar...,PREMIUM,"Al Marwa Towers, Cornich Al Buhaira, Sharjah",1575000.0,...,34484d29-d281-4286-bee6-0c8d3bfcdb3b,25.325766,55.391755,NaN,Large,"(1000000.0, 2000000.0]",NaN,Listed 1 month ago,30.0,Recently Listed
1892,harringtons-4726134,Apartment,UNVERIFIED,2,2,"1,271",Multiple Options | No Commission | High ROI,PREMIUM,"Gemini Splendor, Mohammed Bin Rashid City, Dubai",2032000.0,...,650196de-5f79-4b55-89d7-6e93562a4af6,25.166376,55.276657,NaN,Medium,"(2000000.0, 3000000.0]",NaN,Listed 26 days ago,26.0,Recently Listed


In [49]:
df.drop(['Unnamed: 17', 'Listed on', 'Days Listed'], axis=1, inplace=True)


In [51]:
!pip install geopy

     ------------------------------------ 119.8/119.8 kB 500.4 kB/s eta 0:00:00
     -------------------------------------- 40.3/40.3 kB 485.2 kB/s eta 0:00:00


In [52]:
import geopy
from geopy.geocoders import Nominatim

# Create a geocoder instance
geolocator = Nominatim(user_agent="property-segmentation")

# Define a function to get the city or emirate based on latitude and longitude
def get_city_from_coordinates(latitude, longitude):
    location = geolocator.reverse((latitude, longitude), exactly_one=True)
    if location:
        address = location.raw['address']
        city = address.get('city', '')
        emirate = address.get('state', '')
        return city, emirate
    return None, None

# Apply the function to create new columns for city and emirate
df['City'], df['Emirate'] = zip(*df.apply(
    lambda row: get_city_from_coordinates(row['Latitude'], row['Longitude']), axis=1
))


In [54]:
# Create a mapping dictionary for Arabic to English names
arabic_to_english = {
    "دبي": "Dubai",
    "أبوظبي": "Abu Dhabi",
    "الشارقة": "Sharjah",
    # Add more mappings for other cities and emirates
}

# Create new columns for English city and emirate names
df["City (English)"] = ""
df["Emirate (English)"] = ""

# Function to map Arabic names to English names
def map_arabic_to_english(arabic_name):
    if arabic_name in arabic_to_english:
        return arabic_to_english[arabic_name]
    else:
        return "Unknown"

# Map Arabic names to English names for city and emirate columns
df["City (English)"] = df["City"].apply(map_arabic_to_english)
df["Emirate (English)"] = df["Emirate"].apply(map_arabic_to_english)


In [57]:
# Define the geographical zones
zones = {
    "Coastal Area": {"latitude_min": 24.5, "latitude_max": 25.5, "longitude_min": 54.5, "longitude_max": 55.5},
    "Desert Region": {"latitude_min": 24.0, "latitude_max": 25.0, "longitude_min": 55.5, "longitude_max": 56.5},
    "Mountainous Area": {"latitude_min": 25.0, "latitude_max": 26.0, "longitude_min": 55.0, "longitude_max": 56.0}
}

# Create a new column for geographical zone
df["Geographical Zone"] = ""

# Function to assign properties to zones
def assign_zone(latitude, longitude):
    for zone, values in zones.items():
        if (values["latitude_min"] <= latitude <= values["latitude_max"]) and \
           (values["longitude_min"] <= longitude <= values["longitude_max"]):
            return zone
    return "Unknown"

# Assign zones to properties based on latitude and longitude
for index, row in df.iterrows():
    latitude = float(row["Latitude"])
    longitude = float(row["Longitude"])
    zone = assign_zone(latitude, longitude)
    df.at[index, "Geographical Zone"] = zone


In [58]:
df

,Reference,Property Type,Ad Type,Bedrooms,Bathrooms,Area (sqft),Details,Service Type,Address,Price (AED),...,Longitude,Location Segment,Area Segment,Price Segment,Listed Segmentation,City,Emirate,City (English),Emirate (English),Geographical Zone
0,insight-4993939,Apartment,VERIFIED,1,1,843,WITH BALCONY | 1B/R | SKYCOURT | POOL VIEW,PREMIUM,"Skycourts Tower F, Skycourts Towers, Dubai Lan...",380000.0,...,55.385788,NaN,Small,"(0.0, 1000000.0]",Recently Listed,,دبي,Unknown,Dubai,Coastal Area
1,WIL-S-5498,Villa,VERIFIED,2,2,2967,2 BEDROOM | EXTENDED & UPGRADED | CORNER PLOT,PREMIUM,"Mediterranean Villas, Jumeirah Village Triangl...",3300000.0,...,55.190023,Jumeirah Village Triangle,Large,"(3000000.0, inf]",Listed more than 6 months ago,,دبي,Unknown,Dubai,Coastal Area
2,luxehomes-4946277,Apartment,VERIFIED,2,3,1709,2-Bedroom + Study | Vacant | Palm Sea View,PREMIUM,"Oceana Atlantic, Oceana, Palm Jumeirah, Dubai",5500000.0,...,55.137291,Palm Jumeirah,Medium,"(3000000.0, inf]",Recently Listed,,دبي,Unknown,Dubai,Coastal Area
3,PB-S-1641,Townhouse,VERIFIED,3,4,2235,3 Bed Type 1 | Single Row | Large Plot | Rented,PREMIUM,"Hayat Townhouses, Town Square, Dubai",2000000.0,...,55.298557,NaN,Large,"(1000000.0, 2000000.0]",Recently Listed,,دبي,Unknown,Dubai,Coastal Area
4,eandt-4966526,Villa,VERIFIED,4,4,2445,Serene desert view | Single Row | Bright,PREMIUM,"Sun, Arabian Ranches 3, Dubai",3000000.0,...,55.321369,NaN,Large,"(2000000.0, 3000000.0]",Recently Listed,,دبي,Unknown,Dubai,Coastal Area
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1889,ENRE-FA-CT-S-0032,Apartment,VERIFIED,1,2,875,1 Bed | Stunning Views | Prime Location,PREMIUM,"Cayan Tower, Dubai Marina, Dubai",1950000.0,...,55.145145,Palm Jumeirah,Small,"(1000000.0, 2000000.0]",Recently Listed,,دبي,Unknown,Dubai,Coastal Area
1890,SALE015290,Apartment,UNVERIFIED,2,3,"2,237",Marina Views / Upgraded / Stunning,PREMIUM,"Sadaf 4, Sadaf, Jumeirah Beach Residence, Dubai",3900000.0,...,55.137152,Palm Jumeirah,Large,"(3000000.0, inf]",Newly Listed,,دبي,Unknown,Dubai,Coastal Area
1891,MZY-S-2061,Apartment,UNVERIFIED,4,5,"3,100",Sea view Full Upgraded 4BHK-2 Parking - Al Mar...,PREMIUM,"Al Marwa Towers, Cornich Al Buhaira, Sharjah",1575000.0,...,55.391755,NaN,Large,"(1000000.0, 2000000.0]",Recently Listed,مدينة الشارقة,الشارقة,Unknown,Sharjah,Coastal Area
1892,harringtons-4726134,Apartment,UNVERIFIED,2,2,"1,271",Multiple Options | No Commission | High ROI,PREMIUM,"Gemini Splendor, Mohammed Bin Rashid City, Dubai",2032000.0,...,55.276657,NaN,Medium,"(2000000.0, 3000000.0]",Recently Listed,,دبي,Unknown,Dubai,Coastal Area


In [59]:
df.drop(['Location Segment', 'City', 'Emirate', 'City (English)'], axis=1, inplace=True)


In [61]:
df.drop(['Details'], axis=1, inplace=True)


In [63]:
df['Location'] = df['Address'].str.split(',').str[-2].str.strip()

print(df['Area'])

0                      Dubai Land
1       Jumeirah Village Triangle
2                   Palm Jumeirah
3                     Town Square
4               Arabian Ranches 3
                  ...            
1889                 Dubai Marina
1890     Jumeirah Beach Residence
1891           Cornich Al Buhaira
1892     Mohammed Bin Rashid City
1893                        Mudon
Name: Location, Length: 1894, dtype: object


,Reference,Property Type,Ad Type,Bedrooms,Bathrooms,Area (sqft),Service Type,Address,Price (AED),DLD Permit Number,Agent ID,Latitude,Longitude,Area Segment,Price Segment,Listed Segmentation,Emirate (English),Geographical Zone,Location
0,insight-4993939,Apartment,VERIFIED,1,1,843,PREMIUM,"Skycourts Tower F, Skycourts Towers, Dubai Lan...",380000.0,7.129145e+09,2f2d1393-2c25-40fa-9cf7-f5b0801f0075,25.090079,55.385788,Small,"(0.0, 1000000.0]",Recently Listed,Dubai,Coastal Area,Dubai Land
1,WIL-S-5498,Villa,VERIFIED,2,2,2967,PREMIUM,"Mediterranean Villas, Jumeirah Village Triangl...",3300000.0,6.511934e+09,ca212376-24d4-42ff-a81b-8d9c2933ac0c,25.047288,55.190023,Large,"(3000000.0, inf]",Listed more than 6 months ago,Dubai,Coastal Area,Jumeirah Village Triangle
2,luxehomes-4946277,Apartment,VERIFIED,2,3,1709,PREMIUM,"Oceana Atlantic, Oceana, Palm Jumeirah, Dubai",5500000.0,7.125917e+09,ebee2af7-7b22-4d11-9746-8e15411f5ff6,25.111400,55.137291,Medium,"(3000000.0, inf]",Recently Listed,Dubai,Coastal Area,Palm Jumeirah
3,PB-S-1641,Townhouse,VERIFIED,3,4,2235,PREMIUM,"Hayat Townhouses, Town Square, Dubai",2000000.0,6.919433e+09,22be22d1-1ee6-4eb6-a6b0-603f07356406,25.001965,55.298557,Large,"(1000000.0, 2000000.0]",Recently Listed,Dubai,Coastal Area,Town Square
4,eandt-4966526,Villa,VERIFIED,4,4,2445,PREMIUM,"Sun, Arabian Ranches 3, Dubai",3000000.0,6.924877e+10,97f12ec3-dc4a-4a74-95c5-f489361cefcf,25.068276,55.321369,Large,"(2000000.0, 3000000.0]",Recently Listed,Dubai,Coastal Area,Arabian Ranches 3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1889,ENRE-FA-CT-S-0032,Apartment,VERIFIED,1,2,875,PREMIUM,"Cayan Tower, Dubai Marina, Dubai",1950000.0,7.110258e+09,98d4d654-8f42-44a4-83e1-9655e813c9ae,25.086716,55.145145,Small,"(1000000.0, 2000000.0]",Recently Listed,Dubai,Coastal Area,Dubai Marina
1890,SALE015290,Apartment,UNVERIFIED,2,3,"2,237",PREMIUM,"Sadaf 4, Sadaf, Jumeirah Beach Residence, Dubai",3900000.0,7.115745e+09,4ca11738-4a26-4ccf-ba18-913cc80e0131,25.080577,55.137152,Large,"(3000000.0, inf]",Newly Listed,Dubai,Coastal Area,Jumeirah Beach Residence
1891,MZY-S-2061,Apartment,UNVERIFIED,4,5,"3,100",PREMIUM,"Al Marwa Towers, Cornich Al Buhaira, Sharjah",1575000.0,NaN,34484d29-d281-4286-bee6-0c8d3bfcdb3b,25.325766,55.391755,Large,"(1000000.0, 2000000.0]",Recently Listed,Sharjah,Coastal Area,Cornich Al Buhaira
1892,harringtons-4726134,Apartment,UNVERIFIED,2,2,"1,271",PREMIUM,"Gemini Splendor, Mohammed Bin Rashid City, Dubai",2032000.0,7.132608e+09,650196de-5f79-4b55-89d7-6e93562a4af6,25.166376,55.276657,Medium,"(2000000.0, 3000000.0]",Recently Listed,Dubai,Coastal Area,Mohammed Bin Rashid City


In [72]:
price_labels = {
    '0.0, 1000000.0': '<1M',
    '1000000.0, 2000000.0': '1M-2M',
    '2000000.0, 3000000.0': '2M-3M',
    '3000000.0, inf': '>3M'
}

df['Price Segment'] = df['Price Segment'].replace(price_labels)

df

,Reference,Property Type,Ad Type,Bedrooms,Bathrooms,Area (sqft),Service Type,Address,Price (AED),DLD Permit Number,Agent ID,Latitude,Longitude,Area Segment,Price Segment,Listed Segmentation,Emirate (English),Geographical Zone,Location
0,insight-4993939,Apartment,VERIFIED,1,1,843,PREMIUM,"Skycourts Tower F, Skycourts Towers, Dubai Lan...",380000.0,7.129145e+09,2f2d1393-2c25-40fa-9cf7-f5b0801f0075,25.090079,55.385788,Small,<1M,Recently Listed,Dubai,Coastal Area,Dubai Land
1,WIL-S-5498,Villa,VERIFIED,2,2,2967,PREMIUM,"Mediterranean Villas, Jumeirah Village Triangl...",3300000.0,6.511934e+09,ca212376-24d4-42ff-a81b-8d9c2933ac0c,25.047288,55.190023,Large,>3M,Listed more than 6 months ago,Dubai,Coastal Area,Jumeirah Village Triangle
2,luxehomes-4946277,Apartment,VERIFIED,2,3,1709,PREMIUM,"Oceana Atlantic, Oceana, Palm Jumeirah, Dubai",5500000.0,7.125917e+09,ebee2af7-7b22-4d11-9746-8e15411f5ff6,25.111400,55.137291,Medium,>3M,Recently Listed,Dubai,Coastal Area,Palm Jumeirah
3,PB-S-1641,Townhouse,VERIFIED,3,4,2235,PREMIUM,"Hayat Townhouses, Town Square, Dubai",2000000.0,6.919433e+09,22be22d1-1ee6-4eb6-a6b0-603f07356406,25.001965,55.298557,Large,1M-2M,Recently Listed,Dubai,Coastal Area,Town Square
4,eandt-4966526,Villa,VERIFIED,4,4,2445,PREMIUM,"Sun, Arabian Ranches 3, Dubai",3000000.0,6.924877e+10,97f12ec3-dc4a-4a74-95c5-f489361cefcf,25.068276,55.321369,Large,2M-3M,Recently Listed,Dubai,Coastal Area,Arabian Ranches 3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1889,ENRE-FA-CT-S-0032,Apartment,VERIFIED,1,2,875,PREMIUM,"Cayan Tower, Dubai Marina, Dubai",1950000.0,7.110258e+09,98d4d654-8f42-44a4-83e1-9655e813c9ae,25.086716,55.145145,Small,1M-2M,Recently Listed,Dubai,Coastal Area,Dubai Marina
1890,SALE015290,Apartment,UNVERIFIED,2,3,"2,237",PREMIUM,"Sadaf 4, Sadaf, Jumeirah Beach Residence, Dubai",3900000.0,7.115745e+09,4ca11738-4a26-4ccf-ba18-913cc80e0131,25.080577,55.137152,Large,>3M,Newly Listed,Dubai,Coastal Area,Jumeirah Beach Residence
1891,MZY-S-2061,Apartment,UNVERIFIED,4,5,"3,100",PREMIUM,"Al Marwa Towers, Cornich Al Buhaira, Sharjah",1575000.0,NaN,34484d29-d281-4286-bee6-0c8d3bfcdb3b,25.325766,55.391755,Large,1M-2M,Recently Listed,Sharjah,Coastal Area,Cornich Al Buhaira
1892,harringtons-4726134,Apartment,UNVERIFIED,2,2,"1,271",PREMIUM,"Gemini Splendor, Mohammed Bin Rashid City, Dubai",2032000.0,7.132608e+09,650196de-5f79-4b55-89d7-6e93562a4af6,25.166376,55.276657,Medium,2M-3M,Recently Listed,Dubai,Coastal Area,Mohammed Bin Rashid City


In [64]:
df['Price Segment'] = df['Price Segment'].str.replace('(', '').str.replace('[', '').str.replace(')', '').str.replace(']', '')

print(df['Price Segment'])

0             0.0, 1000000.0
1             3000000.0, inf
2             3000000.0, inf
3       1000000.0, 2000000.0
4       2000000.0, 3000000.0
                ...         
1889    1000000.0, 2000000.0
1890          3000000.0, inf
1891    1000000.0, 2000000.0
1892    2000000.0, 3000000.0
1893    2000000.0, 3000000.0
Name: Price Segment, Length: 1894, dtype: object


C:\Users\mmdip\AppData\Local\Temp\ipykernel_26060\3625804692.py:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df['Price Segment'] = df['Price Segment'].str.replace('(', '').str.replace('[', '').str.replace(')', '').str.replace(']', '')


In [68]:
len(df)

1894

In [73]:
df

,Reference,Property Type,Ad Type,Bedrooms,Bathrooms,Area (sqft),Service Type,Address,Price (AED),DLD Permit Number,Agent ID,Latitude,Longitude,Area Segment,Price Segment,Listed Segmentation,Emirate (English),Geographical Zone,Location
0,insight-4993939,Apartment,VERIFIED,1,1,843,PREMIUM,"Skycourts Tower F, Skycourts Towers, Dubai Lan...",380000.0,7.129145e+09,2f2d1393-2c25-40fa-9cf7-f5b0801f0075,25.090079,55.385788,Small,<1M,Recently Listed,Dubai,Coastal Area,Dubai Land
1,WIL-S-5498,Villa,VERIFIED,2,2,2967,PREMIUM,"Mediterranean Villas, Jumeirah Village Triangl...",3300000.0,6.511934e+09,ca212376-24d4-42ff-a81b-8d9c2933ac0c,25.047288,55.190023,Large,>3M,Listed more than 6 months ago,Dubai,Coastal Area,Jumeirah Village Triangle
2,luxehomes-4946277,Apartment,VERIFIED,2,3,1709,PREMIUM,"Oceana Atlantic, Oceana, Palm Jumeirah, Dubai",5500000.0,7.125917e+09,ebee2af7-7b22-4d11-9746-8e15411f5ff6,25.111400,55.137291,Medium,>3M,Recently Listed,Dubai,Coastal Area,Palm Jumeirah
3,PB-S-1641,Townhouse,VERIFIED,3,4,2235,PREMIUM,"Hayat Townhouses, Town Square, Dubai",2000000.0,6.919433e+09,22be22d1-1ee6-4eb6-a6b0-603f07356406,25.001965,55.298557,Large,1M-2M,Recently Listed,Dubai,Coastal Area,Town Square
4,eandt-4966526,Villa,VERIFIED,4,4,2445,PREMIUM,"Sun, Arabian Ranches 3, Dubai",3000000.0,6.924877e+10,97f12ec3-dc4a-4a74-95c5-f489361cefcf,25.068276,55.321369,Large,2M-3M,Recently Listed,Dubai,Coastal Area,Arabian Ranches 3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1889,ENRE-FA-CT-S-0032,Apartment,VERIFIED,1,2,875,PREMIUM,"Cayan Tower, Dubai Marina, Dubai",1950000.0,7.110258e+09,98d4d654-8f42-44a4-83e1-9655e813c9ae,25.086716,55.145145,Small,1M-2M,Recently Listed,Dubai,Coastal Area,Dubai Marina
1890,SALE015290,Apartment,UNVERIFIED,2,3,"2,237",PREMIUM,"Sadaf 4, Sadaf, Jumeirah Beach Residence, Dubai",3900000.0,7.115745e+09,4ca11738-4a26-4ccf-ba18-913cc80e0131,25.080577,55.137152,Large,>3M,Newly Listed,Dubai,Coastal Area,Jumeirah Beach Residence
1891,MZY-S-2061,Apartment,UNVERIFIED,4,5,"3,100",PREMIUM,"Al Marwa Towers, Cornich Al Buhaira, Sharjah",1575000.0,NaN,34484d29-d281-4286-bee6-0c8d3bfcdb3b,25.325766,55.391755,Large,1M-2M,Recently Listed,Sharjah,Coastal Area,Cornich Al Buhaira
1892,harringtons-4726134,Apartment,UNVERIFIED,2,2,"1,271",PREMIUM,"Gemini Splendor, Mohammed Bin Rashid City, Dubai",2032000.0,7.132608e+09,650196de-5f79-4b55-89d7-6e93562a4af6,25.166376,55.276657,Medium,2M-3M,Recently Listed,Dubai,Coastal Area,Mohammed Bin Rashid City


In [74]:
df.to_csv("Proerpties_segmented.csv", index=False)

In [75]:
import folium
from folium.plugins import HeatMap

# Convert 'Price (AED)' column to numeric type
df['Price (AED)'] = pd.to_numeric(df['Price (AED)'], errors='coerce')

# Drop rows with NaN (non-numeric) values in 'Price (AED)'
filtered_properties = df.dropna(subset=['Price (AED)'])

# Create a map centered at Dubai
dubai_map = folium.Map(location=[25.2048, 55.2708], zoom_start=12)

# Create a HeatMap layer using property prices
heat_data = filtered_properties[['Latitude', 'Longitude', 'Price (AED)']].values
HeatMap(heat_data).add_to(dubai_map)

dubai_map


In [98]:
df1 = pd.read_csv(r"C:\Users\mmdip\Dropbox\Python Projects\Scrappin Project\Cleaned Data\Dubai Properties Cleaned Data\properties-amenities.csv")

In [99]:
len(df1)

20380

In [100]:
values_to_drop = ['a6fad45b-8f6c-47dd-9ae8-a79d99a614b2', 'dae1dc3a-a149-48de-aec7-919d0413c6ce', '6de36bba-8208-4a01-a28a-c029d1c01469']
df1 = df1[~df1['Amenity ID'].isin(values_to_drop)].reset_index(drop=True)
len(df1)

15464

In [95]:
df1.head(10)

,Reference,Amenity ID
0,insight-4993939,d0df0df6-306b-486f-b1a9-d7bde5fac86a
1,insight-4993939,92957fc0-d7f8-49d8-b17b-2cba2f69304e
2,insight-4993939,49812bc9-7de2-428e-aeba-8af02809435d
3,insight-4993939,7bc14746-3b99-46da-9184-d6510f655a8a
4,insight-4993939,e7cf3885-6463-402d-adeb-8f92d7f7477a
5,insight-4993939,d03f55b8-da67-44df-a5a0-203c262460a2
6,insight-4993939,a7b5d417-a2e6-4086-b0fc-fcd12914f532
7,insight-4993939,23acad16-6ca7-4bba-a32a-1f631bc7d94a
8,insight-4993939,5a349f91-fcc0-4474-9e9d-ad322806205f
9,insight-4993939,8fae7faa-e9b9-4816-b186-02c8ad26b257


In [96]:
df1.isnull().sum()

Reference     0
Amenity ID    0
dtype: int64

In [101]:
df1.to_csv("property-amenity-cleaned.csv", index=False)